In [0]:
# INCLUDE_HEADER_TRUE
# INCLUDE_FOOTER_TRUE

# Demo - Automating SCD Type 2 with AUTO CDC in Lakeflow Spark Declarative Pipelines

## Overview 

This demonstration showcases how to implement automated Change Data Capture (CDC) for Slowly Changing Dimension (SCD) Type 2 patterns using Lakeflow Spark Declarative Pipelines. 

Learners will learn to build an end-to-end streaming pipeline that automatically processes customer data changes (inserts, updates, and deletes) while maintaining complete historical records. The demo uses real-world retail customer data from Databricks Marketplace and demonstrates how AUTO CDC INTO simplifies complex CDC operations that traditionally required manual coding. 

Learners will create a multi-layered architecture (Bronze → Silver → Gold) with automated data quality checks, incremental processing, and materialized views for both current and historical customer analytics.

### Learning Objectives
By the end of this demonstration, you will be able to:

- **Configure and implement AUTO CDC INTO** for automated Change Data Capture processing in Lakeflow Spark Declarative Pipelines with SCD Type 2 patterns

- **Design multi-layered data architectures** using Bronze (raw ingestion), Silver (CDC processing), and Gold (analytics-ready) layers with streaming tables and materialized views

- **Apply comprehensive data quality constraints** using pipeline expectations with WARN, DROP, and FAIL actions to ensure data integrity across CDC operations

- **Process streaming JSON files incrementally** using Auto Loader and manage customer lifecycle events (INSERT, UPDATE, DELETE) while preserving historical versions

- **Create analytics ready materialized views** that automatically maintain current customer states and track removed customers for downstream reporting and business intelligence


### CDC Pipeline Overview

In this demonstration, you'll build a Lakeflow Spark Declarative Pipeline that performs the full medallion flow from raw ingestion to curated analytics:

1. **Ingest JSON source files** from cloud storage into a **Bronze raw streaming table** using Auto Loader.  
2. **Apply data quality expectations** to the raw table to produce a **clean Bronze table** ready for downstream processing.  
3. **Implement CDC SCD Type 2** logic with `AUTO CDC INTO` to maintain an up-to-date **Silver customer table** with full change history.  
4. **Create Gold materialized views** for **current customers** and **deleted customers**, enabling simplified analytics and reporting.

![CDC Pipeline Overview](./Includes/images/cdc_lecture/cdc_pipeline_overview.png)

## REQUIRED - SELECT SERVERLESS COMPUTE VERSION 4

This demonstration was developed using **Serverless v4**. 
- Selecting a serverless version: [Select an environment version](https://docs.databricks.com/aws/en/compute/serverless/dependencies#-select-an-environment-version)

**NOTE:** While all purpose compute or other **Serverless** versions may work, they were not tested.

## A. Classroom Setup
Follow the cells below to set up your Workspace for the demonstration.

### A1. Access Marketplace Data

##### NOTE: If you are running this lab in **Vocareum** as a **Partner**, the share is already installed and available as **dbacademy_retail**. Please use this as the value for the `your_marketplace_share_catalog_name` variable below.

If you are running this in your own Workspace, complete the following steps to get your own copy of the Marketplace data:

1. Open **Databricks Marketplace** in a new tab.  

2. Search for `Simulated Retail Customer Data`.  

3. Select the tile titled **Simulated Retail Customer Data (Databricks provided)**.  

4. Click **Get instant access**.  

5. **Enter a unique catalog name** for your share to avoid receiving a duplicate catalog error in shared Workspaces. For example: `dbacademy_retail_yourname`.  

6. Review and accept the terms, then click **Get instant access** to complete the setup.

7. Update the variable `your_marketplace_share_catalog_name` in cell below to point to your shared catalog from Marketplace.

In [0]:
## Update the variable below to reference your marketplace catalog name

## NOTE: If you are using Vocareum use the value 'dbacademy_retail' catalog below
your_marketplace_share_catalog_name = 'YOUR_MARKETPLACE_SHARE_CATALOG'

### A2. Configure Your Catalog and Schema

1. In this step, you must **specify a catalog that you own or have write access to**.
    - If you already have a catalog, update the code below to use its name.
    - If you do not yet have one, create a catalog first, then return and update the code.

    Replace the value for the `my_catalog` variable with your catalog name.


In [0]:
my_catalog = 'YOUR_CATALOG'   ## <---- Simply replace with your catalog name

2. Run the cells below to setup your demonstration environment. The setup will:

    - Check the variables from above were created
    - Create three schemas in your specified catalog:  
      - **sdp_cdc_1_bronze**
      - **sdp_cdc_2_silver**
      - **sdp_cdc_3_gold**  
    - Creates a volume named **your-catalog.sdp_cdc_1_bronze.customer_source_files** and adds a single JSON file.
    - Checks your specified compute

    This ensures that all schemas, tables and objects are created in your catalog.

In [0]:
%run ./Includes/Classroom-Setup-auto-cdc-demo

In [0]:
my_vol_path = auto_cdc_demo_setup(
    my_catalog = my_catalog, 
    marketplace_catalog = your_marketplace_share_catalog_name, 
    schema = 'sdp_cdc',
    source_volume = 'customer_source_files',
    reset_volume = False ## <-- Set to True to delete all files in your volumes to start fresh if you've already complete the demo
)

2. Confirm the variable `my_vol_path` created in the classroom setup points to **your-catalog.sdp_cdc_1_bronze.customer_source_files** volume path.

    **Example:** `/Volumes/your-catalog/sdp_cdc_1_bronze/customer_source_files`

In [0]:
print(my_vol_path)

## B. Explore the Customer Data Source File(s) in Your Volume

1. Run the cell below to programmatically view the files in your `/Volumes/your-catalog/sdp_cdc_1_bronze/customer_source_files/` volume **path**.
  
    Confirm you only see one file for customers (**00.json**). 

In [0]:
spark.sql(f'LIST "{my_vol_path}"').display()

2. Run the query below to explore the customers **00.json** file located in your **customer_source_files** volume within **your-catalog.sdp_cdc_1_bronze** schema. 

    Note the following:

      a. The file contains **939 customers** (remember this number).

      b. It includes general customer information such as **email**, **name**, and **address**.

      c. The **timestamp** column specifies the logical order (sequence) of customer events in the source data as a UNIX timestamp.

      d. The **operation** column indicates whether the entry is for a new customer, a deletion, or an update.

      - **NOTE:** Since this is the first JSON file, all rows will be considered new customers.

  **NOTE:** The `my_vol_path` SQL variable was created for you in the classroom setup.

In [0]:
%sql
SELECT *
FROM read_files(
  my_vol_path || '/00.json',  -- my_vol_path is the path to your source JSON files in the volume your-catalog.sdp_cdc_1_bronze.customer_source_files
  format => "JSON"
)
ORDER BY operation;

### Question
How can we ingest new raw JSON source files with customer updates into our pipeline to update a **silver table** when inserts, updates, or deletes occur, while also maintaining historical records (SCD Type 2)?

### Answer
Use `AUTO CDC INTO` with Spark Declarative Pipelines!

View the Databricks documentation [Processing a change data feed: Keep only the latest data vs. keep historical versions of data](https://docs.databricks.com/aws/en/ldp/what-is-change-data-capture#processing-a-change-data-feed-keep-only-the-latest-data-vs-keep-historical-versions-of-data) for more information.

## C. Building the CDC Spark Declarative Pipeline with `AUTO CDC INTO`

### C1. Enable the Lakeflow Pipelines Editor

Complete the following steps to confirm or enable the **Lakeflow Pipelines Editor**:

1. In the top-right corner of the workspace, select your **account icon** ![Account Icon](./Includes/images/account_icon.png) (*Your letter will differ*).  

2. Right-click **Settings** and choose **Open link in new tab**.  
3. In the left sidebar, select **Developer** under **User**.  
4. In the **Experimental features** section, locate **Lakeflow Pipelines Editor** and toggle it **on**.


### C2. Create a Lakeflow Spark Declarative Pipeline using the Lakeflow Pipelines Editor
Complete the following steps to create your Spark Declarative Pipeline:

1. In the main navigation pane, right-click **Jobs & Pipelines** and select **Open link in New Tab**.  

2. In the new tab, select **Create → ETL Pipeline**.  

   **NOTE:** If prompted to **Try the new Lakeflow Pipelines Editor**, choose **Enable Lakeflow Pipelines Editor**. This appears only if you did not complete the previous step.  

3. At the top, complete the following:
   - Name your pipeline `demo_auto_cdc_pipeline_yourname`
   - Select your default **catalog** and **schema**:  
        - **Catalog:** The catalog you specified for this notebook  
        - **Schema:** **sdp_cdc_1_bronze**  

4. Select **Start with an empty file**. In the pop-up window, specify the following:  
   - For **Location where the pipeline folder will be created** - Specify the folder this notebook resides in.
   - Select **SQL**
   - Select **Create**

5. Rename the **transformations** folder to `my_pipeline`.

6. Rename the `my_transformations.sql` file to `cdc_pipeline.sql`.

7. Leave the **Lakeflow Pipelines Editor** page open.

#### Checkpoint
![Create SDP Checkpoint](./Includes/images/cdc_lecture/cdc_create_initial_pipeline.png)

### C3. Configure Pipeline Parameter

1. Run the cell below to get the path to your raw data source JSON files in your **customer_source_files** volume.

In [0]:
print(my_vol_path)

2. Copy the path above and add it as a configuration parameter in your **Spark Declarative Pipeline**. This will reference your **sdp_cdc_1_bronze.customer_source_files** volume that contains your raw JSON file(s) that you explored earlier:

   a. Select **Settings** in your pipeline tab.

   b. Under **Configuration** select **Add configuration**.

   c. For **Key** add `source`

   d. For **Value** add the `path to your volume` from above.

   e. Select **Save**.

   **NOTE:** For more information on configuration parameters, check out the Databricks documentation [Use parameters with Lakeflow Declarative Pipelines](https://docs.databricks.com/aws/en/ldp/parameters)

#### Checkpoint (your path will vary)
<img src="./Includes/images/cdc_lecture/cdc_config_parameter_value.png" alt="Config Parameter Checkpoint" width="600">


### C4. Create Bronze Layer with Auto Loader for Incremental Ingestion

1. In this step, you'll define the **Bronze raw** layer of your **Spark Declarative Pipeline** to create the table: **sdp_cdc_1_bronze.customers_bronze_raw_demo**.  

   This layer will incrementally ingest raw **JSON** files from cloud storage into a **streaming table** using **Auto Loader**.  
   
   Copy the SQL code below and paste it into your `cdc_pipeline.sql` file. 

<button onclick="copyBlock()">Copy to clipboard</button>

<pre id="copy-block" style="font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; border:1px solid #e5e7eb; border-radius:10px; background:#f8fafc; padding:14px 16px; font-size:0.85rem; line-height:1.35; white-space:pre;">
<code>
<!-------------------ADD SOLUTION CODE BELOW------------------->
------------------------------------------------------
-- STEP 1: JSON -> Bronze Ingestion
------------------------------------------------------
-- Ingest the JSON files from cloud storage into a streaming table using Auto Loader
CREATE OR REFRESH STREAMING TABLE sdp_cdc_1_bronze.customers_bronze_raw_demo
  COMMENT "Raw data from customers CDC feed"
AS 
SELECT 
  *,
  current_timestamp() processing_time,  -- Obtain the ingestion processing time for the rows
  _metadata.file_name as source_file    -- Obtain the file name of the record
FROM STREAM read_files(
  "${source}",  -- References your SDP parameter that points to your demonstration volume containing your JSON data source files
  format => "json");
<!-------------------END SOLUTION CODE------------------->
</code></pre>

<script>
function copyBlock() {
  const el = document.getElementById("copy-block");
  if (!el) return;

  const text = el.innerText;

  // Preferred modern API
  if (navigator.clipboard && navigator.clipboard.writeText) {
    navigator.clipboard.writeText(text)
      .then(() => alert("Copied to clipboard"))
      .catch(err => {
        console.error("Clipboard write failed:", err);
        fallbackCopy(text);
      });
  } else {
    fallbackCopy(text);
  }
}

function fallbackCopy(text) {
  const textarea = document.createElement("textarea");
  textarea.value = text;
  textarea.style.position = "fixed";
  textarea.style.left = "-9999px";
  document.body.appendChild(textarea);
  textarea.select();
  try {
    document.execCommand("copy");
    alert("Copied to clipboard");
  } catch (err) {
    console.error("Fallback copy failed:", err);
    alert("Could not copy to clipboard. Please copy manually.");
  } finally {
    document.body.removeChild(textarea);
  }
}
</script>

**Review the code:**

- `CREATE OR REFRESH STREAMING TABLE` - Creates a managed streaming table that automatically updates as new data arrives.  
- `COMMENT` - Describes the purpose of the table for easier discovery and documentation.  
- `current_timestamp()` - Captures the ingestion timestamp for data lineage and auditing.  
- `_metadata.file_name` - Adds the source file name for traceability and debugging.  
- `FROM STREAM read_files()` - Uses Auto Loader to continuously read new JSON files from the specified `${source}` path you added in the previous step. 

2. Select **Dry Run** to verify that your pipeline and SQL code are configured correctly.  
   
   This will validate your settings without actually running the full pipeline.

#### Checkpoint
> **TROUBLESHOOTING:** If the dry run did not work, make sure you configure your `source` pipeline parameter correctly in the **Configure Pipeline Parameter** step above.

<img src="./Includes/images/cdc_lecture/cdc_dry_run_ingest.png" alt="JSON Ingest" width="1100">

### C5. Apply Data Quality Rules to the Bronze Table

With the raw JSON CDC data now streaming into **sdp_cdc_1_bronze.customers_bronze_raw_demo**, we'll add another **bronze** quality layer before moving to **Silver**.  

You'll create an intermediate table **sdp_cdc_1_bronze.customers_bronze_clean** and apply **data quality constraints** to filter and flag invalid records while preserving expected `NULL` values for `DELETE` operations.  

In this step, you'll:  
- Apply **multiple constraints** to check key fields.  
- Use **WARN**, **DROP**, and **FAIL** actions to define how violations are handled.  
- Add **conditional logic** and **regex validation** inside constraints.  

For details, see [Manage data quality with pipeline expectations](https://docs.databricks.com/aws/en/ldp/expectations).


#### C5.1 - About the Data Source

   - The CDC feed contains `INSERT`, `UPDATE`, and `DELETE` operations for customer records. 

   - `INSERT` and `UPDATE` events include valid values for all key columns.  

   - `DELETE` events have `NULL` values for all non-key fields such as **name**, **email**, **address**, **city**, and **state**.  

   **Example of a Dropped Record:**

   | **address** | **city** | **customer_id** | **email** | **name** | **operation** | **state** |
   |--------------|-----------|------------------|-------------|------------|----------------|-------------|
   | `NULL` | `NULL` | `23617` | `NULL` | `NULL` | `DELETE` | `NULL` |


#### C5.2 - Create the Clean Bronze Table Using Pipeline Data Quality Expectations

1. Now that we have a general understanding of the data, let's define the following **data quality constraints** that enforce those rules in a secondary Bronze table named **sdp_cdc_1_bronze.customers_bronze_clean_demo** before the data moves to the Silver stage.

Below is a table of the data quality constraints:

  | **Comment ID** | **Constraint** | **Rule / Expectation** | **Violation Action** | **Purpose** |
  |:------:|----------------|------------------------|----------------------|--------------|
  | **A** | **`valid_id`** | **customer_id** `IS NOT NULL` | **FAIL** | Fails the transaction if a record has a missing **customer_id**. |
  | **B** | **`valid_operation`** | **operation** `IS NOT NULL` | **DROP ROW** | Drops any record missing an **operation** type. |
  | **C** | **`valid_name`** | **name** `IS NOT NULL OR operation = "DELETE"` | **WARN** (default) | Flags missing **name** values unless it's a **DELETE** operation. |
  | **D** | **`valid_address`** | All address fields (**address**, **city**, **state**, **zip_code**) must be `non-null` unless it's a **DELETE** | **WARN** (default) | Ensures complete address information for **INSERT** and **UPDATE** events. |
  | **E** | **`valid_email`** | **email** must match a valid format unless it's a **DELETE** | **DROP ROW** | Uses regex to validate the **email** format and drops invalid records. |


2. Copy the SQL code below and paste it into your `cdc_pipeline.sql` file.  

<button onclick="copyBlock()">Copy to clipboard</button>

<pre id="copy-block" style="font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; border:1px solid #e5e7eb; border-radius:10px; background:#f8fafc; padding:14px 16px; font-size:0.85rem; line-height:1.35; white-space:pre;">
<code>
<!-------------------ADD SOLUTION CODE BELOW------------------->
 ------------------------------------------------------
-- STEP 2: Bronze Raw -> Bronze Clean
------------------------------------------------------
CREATE STREAMING TABLE sdp_cdc_1_bronze.customers_bronze_clean_demo
  (

    -- A. Require a valid customer_id, fail the transaction if missing
    CONSTRAINT valid_id EXPECT (customer_id IS NOT NULL) 
      ON VIOLATION FAIL UPDATE,

    -- B. Require a valid operation, drop any record with NULL operation
    CONSTRAINT valid_operation EXPECT (operation IS NOT NULL) 
      ON VIOLATION DROP ROW,

    -- C. Require name to be present unless the operation is DELETE
    CONSTRAINT valid_name EXPECT (name IS NOT NULL OR operation = "DELETE"),

    -- D. Require full address fields unless operation is DELETE
    CONSTRAINT valid_address EXPECT (
      (address IS NOT NULL 
       AND city IS NOT NULL 
       AND state IS NOT NULL 
       AND zip_code IS NOT NULL)
       OR operation = "DELETE"),

    -- E. Require valid email format (regex), skip check for DELETE, drop invalid rows
    CONSTRAINT valid_email EXPECT (
      rlike(email, '^([a-zA-Z0-9_\\-\\.]+)@([a-zA-Z0-9_\\-\\.]+)\\.([a-zA-Z]{2,5})$') 
      OR operation = "DELETE") 
      ON VIOLATION DROP ROW
  )
  COMMENT "Clean raw bronze data and apply quality constraints"
AS 
SELECT 
  *,
  CAST(from_unixtime(timestamp) AS timestamp) AS timestamp_datetime -- Convert unix timestamp
FROM STREAM sdp_cdc_1_bronze.customers_bronze_raw_demo;
<!-------------------END SOLUTION CODE------------------->
</code></pre>

<script>
function copyBlock() {
  const el = document.getElementById("copy-block");
  if (!el) return;

  const text = el.innerText;

  // Preferred modern API
  if (navigator.clipboard && navigator.clipboard.writeText) {
    navigator.clipboard.writeText(text)
      .then(() => alert("Copied to clipboard"))
      .catch(err => {
        console.error("Clipboard write failed:", err);
        fallbackCopy(text);
      });
  } else {
    fallbackCopy(text);
  }
}

function fallbackCopy(text) {
  const textarea = document.createElement("textarea");
  textarea.value = text;
  textarea.style.position = "fixed";
  textarea.style.left = "-9999px";
  document.body.appendChild(textarea);
  textarea.select();
  try {
    document.execCommand("copy");
    alert("Copied to clipboard");
  } catch (err) {
    console.error("Fallback copy failed:", err);
    alert("Could not copy to clipboard. Please copy manually.");
  } finally {
    document.body.removeChild(textarea);
  }
}
</script>

**Review the code:**

  - This query creates the **sdp_cdc_1_bronze.customers_bronze_clean_demo** table from the **sdp_cdc_1_bronze.customers_bronze_raw_demo** table.  

  - It applies the **data quality constraints** defined above to ensure only valid CDC records progress to the next stage before reaching Silver.

  - Also **Converts UNIX timestamp to readable timestamp** using `CAST(from_unixtime(timestamp) AS timestamp)` as `timestamp_datetime`.

3. Select **Dry Run** to verify that your pipeline and SQL code are configured correctly.  
   
   This will validate your settings without actually running the full pipeline.

#### Checkpoint
<img src="./Includes/images/cdc_lecture/cdc_dry_run_bronze_clean.png" alt="Bronze Clean" width="1200">

### C6.Implement `AUTO CDC INTO` for the Silver Table

Now that the **Bronze** layer (**sdp_cdc_1_bronze.customers_bronze_raw_demo -> sdp_cdc_1_bronze.customers_bronze_clean_demo**) data is clean, we'll create a **Silver** table that automatically applies inserts, updates, and deletes as new changes arrive.  

#### PROBLEM 
Traditionally, handling Change Data Capture (CDC) manually can be complex and error-prone.  

#### SOLUTION 
`AUTO CDC INTO` in **Spark Declarative Pipelines** simplifies this process by automatically managing inserts, updates, and deletes in streaming data, reducing boilerplate code and improving reliability.

`AUTO CDC INTO` provides the following guarantees and requirements:

- Performs incremental and streaming ingestion of CDC data  
- Lets you define one or more primary key fields for identifying records  
- Assumes rows contain inserts and updates by default  
- **Optionally** applies deletes when defined  
- Orders late-arriving records using a **sequencing key** 
- Allows excluding columns with the **`EXCEPT`** keyword  
- Defaults to **SCD Type 1**, though we'll use **SCD Type 2** in this demonstration  


We'll complete this in two steps:

1. **Create an empty Silver target table** to store historical customer data using the SCD Type 2 pattern.  

2. **Use `AUTO CDC INTO`** to automatically apply inserts, updates, and deletes as new changes arrive.


**NOTE:** For more details, see the official documentation: [**The AUTO CDC APIs: Simplify change data capture with Lakeflow Declarative Pipelines**](https://docs.databricks.com/aws/en/ldp/cdc)


#### C6.1 - Create an Empty Silver Target Table
Before we can use AUTO CDC INTO, we need to create the target table that will store our customer data with historical tracking.

1. The SQL command below creates or refreshes a **streaming Silver table** named **sdp_cdc_2_silver.customers_silver_scd2_demo**.  

   This table will continuously receive incremental updates from the **customers_bronze_clean_demo** table as new change data arrives.  

   Copy and paste the code below into your `cdc_pipeline.sql` file.

<button onclick="copyBlock()">Copy to clipboard</button>

<pre id="copy-block" style="font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; border:1px solid #e5e7eb; border-radius:10px; background:#f8fafc; padding:14px 16px; font-size:0.85rem; line-height:1.35; white-space:pre;">
<code>
<!-------------------ADD SOLUTION CODE BELOW------------------->
---------------------------------------------------------------------------------------
-- STEP 3: Processing CDC Data with AUTO CDC INTO
---------------------------------------------------------------------------------------

-- a. Create the streaming target table if it's not already created
CREATE OR REFRESH STREAMING TABLE sdp_cdc_2_silver.customers_silver_scd2_demo
  COMMENT 'SCD Type 2 Historical Customer Data';
<!-------------------END SOLUTION CODE------------------->
</code></pre>

<script>
function copyBlock() {
  const el = document.getElementById("copy-block");
  if (!el) return;

  const text = el.innerText;

  // Preferred modern API
  if (navigator.clipboard && navigator.clipboard.writeText) {
    navigator.clipboard.writeText(text)
      .then(() => alert("Copied to clipboard"))
      .catch(err => {
        console.error("Clipboard write failed:", err);
        fallbackCopy(text);
      });
  } else {
    fallbackCopy(text);
  }
}

function fallbackCopy(text) {
  const textarea = document.createElement("textarea");
  textarea.value = text;
  textarea.style.position = "fixed";
  textarea.style.left = "-9999px";
  document.body.appendChild(textarea);
  textarea.select();
  try {
    document.execCommand("copy");
    alert("Copied to clipboard");
  } catch (err) {
    console.error("Fallback copy failed:", err);
    alert("Could not copy to clipboard. Please copy manually.");
  } finally {
    document.body.removeChild(textarea);
  }
}
</script>

**Review the code**

- `CREATE OR REFRESH STREAMING TABLE` - Creates the table **sdp_cdc_2_silver.customers_silver_scd2_demo** if it doesn't exist, or refreshes it if it does, ensuring it's ready for streaming updates.

- `COMMENT` - Adds a description to document the table's purpose, identifying it as an **SCD Type 2** table used to track historical customer changes over time.

Once the Silver target table exists, we can automatically propagate changes from the cleaned Bronze table using Lakeflow Declarative Pipelines with the `AUTO CDC INTO` command.

#### C6.2 - Perform SCD Type 2 with `AUTO CDC INTO`


1. Before we run `AUTO CDC INTO`, let's take a closer look at the key columns in our cleaned Bronze table (**sdp_cdc_1_bronze.customers_bronze_clean_demo**). 

    Understanding these columns helps explain how **Spark Declarative Pipelines** detects and applies changes across `INSERT`, `UPDATE`, and `DELETE` operations.

    Below is a simplified preview of the data. We've included only the columns that are important for **Change Data Capture (CDC)**.

| **customer_id** (*primary key*) | **name** | **email** | **operation** (*type of change*) | **timestamp_datetime** (*sequence column*) | **source_file** | ... |
|------------------|-----------|------------|----------------|------------------------|------------------|-----|
| 23056 | Brent Chavez | nelsonjoy@example.com | `NEW` | 2021-09-23T17:26:21.000+00:00 | 00.json | ... |
| 23057 | James Cruz | perkinsdeborah@example.net | `UPDATE` | 2021-09-23T18:21:45.000+00:00 | 00.json | ... |
| 23058 | Jennifer Christensen | jmccullough@example.net | `DELETE` | 2021-09-23T00:19:44.000+00:00 | 00.json | ... |
| ... | ... | ... | ... | ... | ... | ... |

**Key Columns**

- **customer_id**- Serves as the **primary key** used to identify each unique customer record. CDC logic depends on this column to determine which row should be `updated` or `deleted` in the target table.

- **operation** - Indicates the type of change from the source system (`NEW`, `UPDATE`, or `DELETE`).  
  - `DELETE` operations remove the corresponding record from the target table.  
  - `NEW` and `UPDATE` operations are automatically processed and merged by `AUTO CDC` using the **customer_id** primary key column.

- **timestamp_datetime** - Defines the **order of operations**. This ensures `updates` are applied in the correct sequence when multiple changes occur for the same record.

- **source_file** - Helps trace which file each record came from, useful for debugging and validation during pipeline runs.

Next, we'll use these columns to perform `AUTO CDC INTO`, allowing Spark Declarative Pipelines **to automatically manage incremental updates between Bronze and Silver**.


2. The `AUTO CDC INTO` command defines a **Lakeflow Declarative Pipeline flow** that applies **SCD Type 2** Change Data Capture logic to keep the **Silver** table continuously up to date.  

   It automatically manages how `INSERTS`, `UPDATES`, and `DELETES` are applied between the following:  

   - `Source:` **sdp_cdc_1_bronze.customers_bronze_clean_demo** - The streaming Bronze table containing cleaned CDC records.  

   - `Target:` **sdp_cdc_2_silver.customers_silver_scd2_demo** - The Silver table that stores the current and historical customer data.


    Copy and paste the code below into your `cdc_pipeline.sql` file.

<button onclick="copyBlock()">Copy to clipboard</button>

<pre id="copy-block" style="font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; border:1px solid #e5e7eb; border-radius:10px; background:#f8fafc; padding:14px 16px; font-size:0.85rem; line-height:1.35; white-space:pre;">
<code>
<!-------------------ADD SOLUTION CODE BELOW------------------->
-- b. Perform SCD Type 2 into the silver table
CREATE FLOW customers_scd_type_2_flow AS 
AUTO CDC INTO sdp_cdc_2_silver.customers_silver_scd2_demo  -- Target: Where processed records are stored
FROM STREAM sdp_cdc_1_bronze.customers_bronze_clean_demo   -- Source: Clean CDC records from Bronze layer
  KEYS (customer_id)                                       -- Primary key: Used to match records for updates/deletes
  APPLY AS DELETE WHEN operation = "DELETE"                -- Delete logic: Remove records marked as DELETE
  SEQUENCE BY timestamp_datetime                           -- Ordering: Ensures changes are applied in correct sequence
  COLUMNS * EXCEPT (timestamp, _rescued_data, operation)   -- Column selection: Include all except metadata fields
  STORED AS SCD TYPE 2;                                    -- SCD Type 2: Maintains historical versions with __START_AT and __END_AT
<!-------------------END SOLUTION CODE------------------->
</code></pre>

<script>
function copyBlock() {
  const el = document.getElementById("copy-block");
  if (!el) return;

  const text = el.innerText;

  // Preferred modern API
  if (navigator.clipboard && navigator.clipboard.writeText) {
    navigator.clipboard.writeText(text)
      .then(() => alert("Copied to clipboard"))
      .catch(err => {
        console.error("Clipboard write failed:", err);
        fallbackCopy(text);
      });
  } else {
    fallbackCopy(text);
  }
}

function fallbackCopy(text) {
  const textarea = document.createElement("textarea");
  textarea.value = text;
  textarea.style.position = "fixed";
  textarea.style.left = "-9999px";
  document.body.appendChild(textarea);
  textarea.select();
  try {
    document.execCommand("copy");
    alert("Copied to clipboard");
  } catch (err) {
    console.error("Fallback copy failed:", err);
    alert("Could not copy to clipboard. Please copy manually.");
  } finally {
    document.body.removeChild(textarea);
  }
}
</script>

**Review the code**

- `CREATE FLOW customers_scd_type_2_flow AS` - Creates a named flow (`customers_scd_type_2_flow`) that defines how CDC changes will be processed.  
- `AUTO CDC INTO sdp_cdc_2_silver.customers_silver_scd2_demo` - Applies the CDC logic to the **target** Silver table.  
- `FROM STREAM sdp_cdc_1_bronze.customers_bronze_clean_demo` - Reads the streaming **source data** that includes new inserts, updates, and deletes.  
- `KEYS (customer_id)` - Identifies the unique customer record by its primary key.  
- `APPLY AS DELETE WHEN operation = "DELETE"` - Ensures records with a delete operation are removed from the target.  
- `SEQUENCE BY timestamp_datetime` - Orders incoming records so late-arriving data is processed correctly.  
- `COLUMNS * EXCEPT (timestamp, _rescued_data, operation)` - Selects all columns from the source except metadata or system fields.  
- `STORED AS SCD TYPE 2` - Specifies the **Slowly Changing Dimension Type 2** method, which updates records in place, keeping historical versions.

- **NOTE:** For more information view the Databricks documentation [AUTO CDC INTO (Lakeflow Declarative Pipelines)](https://docs.databricks.com/aws/en/ldp/developer/ldp-sql-ref-apply-changes-into).

3. Select **Dry Run** to verify that your pipeline and SQL code are configured correctly.  
   
   This will validate your settings without actually running the full pipeline.

#### Checkpoint
<img src="./Includes/images/cdc_lecture/cdc_dry_run_auto_cdc_run1.png" alt="AUTO CDC" width="1200">

## D. Pipeline Execution and Analysis (Run with 1 JSON File)

### D1. Run the Spark Declarative Pipeline

1. In the **Lakeflow Pipelines Editor**, run your **Spark Declarative Pipeline**. The first run may take a few minutes.  

2. Once the pipeline finishes, verify it looks similar to the example below.

#### Checkpoint

<img src="./Includes/images/cdc_lecture/cdc_01_pipeline-run.png" alt="Pipeline Run 1" width="1200">

### D2. Quick Review: AUTO CDC INTO (SCD Type 2 Behavior)


<img src="./Includes/images/cdc_lecture/02-scd-type-2-01-review-slide.png" alt="SCD Type 2 Review" width="1200">

Recall `AUTO CDC INTO` in Spark Declarative Pipelines automatically tracks historical changes without manual merge logic. With **SCD Type 2** it creates two new columns:

- **__START_AT** - Timestamp when the record becomes active. 

- **__END_AT** - Timestamp when the record is closed (after an update or delete).  

When a row changes, the old version gets an **__END_AT** timestamp, and a new version starts with a fresh **__START_AT**. 

This keeps a full SCD Type 2 history automatically.




### D3. Explore the Spark Declarative Pipeline Results

After the pipeline completes, review the outputs to confirm data flow and CDC behavior on the first run with **one** JSON file.

1. In the pipeline graph:
   - **939** rows were streamed through all layers, passing Bronze data quality checks and upserting into **sdp_cdc_2_silver.customers_silver_scd2_demo**.

2. In the **Lakeflow Pipeline Editor** table window:
   - The **Upserted records** column in **sdp_cdc_2_silver.customers_silver_scd2_demo** confirms all **939** rows were inserted as new customers.

3. In the **Tables** view, select **customers_silver_scd2_demo**:
   - Preview the table to inspect the data.
   - Scroll right to see the **__START_AT** and **__END_AT** columns  were automatically added by the `AUTO CDC INTO` SCD Type 2 process indicating active, inactive and removed data (right now all rows are active - **__END_AT** = `null`).


### D4. Explore the Pipeline CDC Silver Streaming Table

1. Run the cell below to confirm that `current_catalog()` references your catalog and that your settings are still active (they might have been cleared).  

    If they were cleared, reset your default catalog using: `USE CATALOG your_catalog_name;`

    

In [0]:
%sql
SELECT current_catalog() AS `Your Current Catalog`

2. Run the query below to view the **sdp_cdc_2_silver.customers_silver_scd2_demo** CDC streaming table (SCD Type 2).  

Observe the following:

- The table contains all **939 rows** from the **00.json** file, as all customers are new.  

- Scroll to the right to see that `AUTO CDC INTO (SCD Type 2)` added two columns:  
  - **__START_AT**: Timestamp showing when the current version of each record became active. This matches the sequence column **timestamp_datetime**.  
  - **__END_AT**: 
    - Timestamp showing when the record became inactive due to a **DELETE** or **UPDATE**. 
    - Filtering where **__END_AT** is `NULL` returns all active customers.  

Since this is the first ingestion from **00.json**, all records are active and **__END_AT** contains only `NULL` values.

**NOTES:**  
- These columns implement **Slowly Changing Dimension (SCD) Type 2** tracking.  
- `AUTO CDC INTO` automatically manages **__START_AT** and **__END_AT** to record the valid time range of each version.  



In [0]:
%sql
SELECT customer_id, email, timestamp_datetime, processing_time, source_file, __START_AT, __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo;

2. Run the cell below to query the CDC table for **all current customers** by filtering where **__END_AT** is `NULL`. 

   Recall a `NULL` value in the **__END_AT** column indicates all active (current) records.

   Review the results and confirm that on the first run all **939** records are active.

In [0]:
%sql
SELECT customer_id, email, timestamp_datetime, processing_time, source_file, __START_AT, __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
WHERE __END_AT IS NULL;    -- Find all current customers

2. To find all **deleted customer records** in an SCD Type 2 table created with `AUTO CDC INTO`, you need to locate the **most recent version of each record** and check if the **__END_AT** `IS NOT NULL` (contains a value).  

    - **NOTE:** Recall a non-null **__END_AT** value indicates that the record has been marked as deleted.  

    The example query below demonstrates one way to find these records. Run the cell and confirm that **0** results are returned, since no deletions have occurred yet in the **sdp_cdc_2_silver.customers_silver_scd2_demo** CDC table. 


**Code details**
- This identifies **removed customers** by grouping all versions, selecting each customer's latest record, and keeping only those marked as deleted, showing their final details before removal.

    - **Use MAX_BY to find the latest record** - The function `MAX_BY(value, __START_AT)` returns the value from the row with the most recent `__START_AT` timestamp.  
        - `MAX_BY(name, __START_AT)` - most recent name  
        - `MAX_BY(address, __START_AT)` - most recent address  
        - `MAX_BY(__END_AT, __START_AT)` - most recent deletion marker from the most recent record
        - etc.

    - `GROUP BY customer_id` - Collects all historical versions of each customer into a single group so we can analyze changes over time.

    - `HAVING MAX_BY(__END_AT, __START_AT) IS NOT NULL` - Condition keeps only customers whose latest record has a value for `__END_AT`, meaning they have been removed.

    - [max_by aggregate function](https://docs.databricks.com/aws/en/sql/language-manual/functions/max_by)



In [0]:
%sql
-- Currently this query will return 0 results since no customers were marked as deleted
SELECT
  customer_id,
  MAX_BY(name, __START_AT)      AS name,
  MAX_BY(address, __START_AT)   AS address,
  MAX_BY(city, __START_AT) AS city,
  MAX_BY(state, __START_AT)   AS state,
  MAX_BY(zip_code, __START_AT)   AS zip_code,
  MAX_BY(__START_AT, __START_AT) AS __START_AT,
  MAX_BY(__END_AT, __START_AT)  AS __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
GROUP BY customer_id
HAVING MAX_BY(__END_AT, __START_AT) IS NOT NULL;

## E. Create Gold Materialized Views

Rather than having users manually query the CDC table to **find current or deleted customers**, you can create **Gold materialized views** that surface this information automatically as an object.  

These views simplify access for business users and can be added directly to your **Lakeflow Declarative Pipeline** for continuous updates (incremental if possible).

In this step we'll create two views:

   - a. **current_customers_gold_demo** - Returns only the most recent, active customer records (where `__END_AT IS NULL`).  

   - b. **removed_customers_gold_demo** - Creates a **Gold Materialized View** that lists all customers who have been **removed (deleted)** from the SCD Type 2 table.  (It uses `MAX_BY` on multiple columns like `name`, `address`, `city`, `state`, and `zip_code` to return the most recent record per customer where the latest `__END_AT` value is not `NULL`.)

**NOTES:**

- Materialized views in **Lakeflow Declarative Pipelines** provide an automatically updated layer for reporting and dashboards. They make it easy to query only the **current customer state** or identify **removed customers** without scanning the entire SCD history.

- When possible, **materialized views** are incrementally updated. For more information, see the [Incremental refresh for materialized views](https://docs.databricks.com/aws/en/optimizations/incremental-refresh).






### E1. Create the Current Customers Materialized View

1. Copy and paste the code below into your `cdc_pipeline.sql` file to create your gold materialized views. Explore the code.

<button onclick="copyBlock()">Copy to clipboard</button>

<pre id="copy-block" style="font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; border:1px solid #e5e7eb; border-radius:10px; background:#f8fafc; padding:14px 16px; font-size:0.85rem; line-height:1.35; white-space:pre;">
<code>
<!-------------------ADD SOLUTION CODE BELOW------------------->
---------------------------------------------------------------------------------------
-- STEP 4: Create Materialized View for Current (Active) Customers
---------------------------------------------------------------------------------------

-- a. Create Gold Materialized View for Current Customers
CREATE OR REFRESH MATERIALIZED VIEW sdp_cdc_3_gold.current_customers_gold_demo
COMMENT "Current updated list of active customers"
AS 
SELECT 
  * EXCEPT (processing_time),
  current_timestamp() updated_at
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
WHERE `__END_AT` IS NULL;      -- Filter for only rows that contain a null value for __END_AT, which indicates the current version of the record
<!-------------------END SOLUTION CODE------------------->
</code></pre>

<script>
function copyBlock() {
  const el = document.getElementById("copy-block");
  if (!el) return;

  const text = el.innerText;

  // Preferred modern API
  if (navigator.clipboard && navigator.clipboard.writeText) {
    navigator.clipboard.writeText(text)
      .then(() => alert("Copied to clipboard"))
      .catch(err => {
        console.error("Clipboard write failed:", err);
        fallbackCopy(text);
      });
  } else {
    fallbackCopy(text);
  }
}

function fallbackCopy(text) {
  const textarea = document.createElement("textarea");
  textarea.value = text;
  textarea.style.position = "fixed";
  textarea.style.left = "-9999px";
  document.body.appendChild(textarea);
  textarea.select();
  try {
    document.execCommand("copy");
    alert("Copied to clipboard");
  } catch (err) {
    console.error("Fallback copy failed:", err);
    alert("Could not copy to clipboard. Please copy manually.");
  } finally {
    document.body.removeChild(textarea);
  }
}
</script>

### E2. Create the Deleted Customers Materialized View

1. Copy and paste the code below into your `cdc_pipeline.sql` file to create your gold materialized views. Explore the code.

- **NOTE:** Because this uses `GROUP BY` with aggregates instead of window functions, Lakeflow can update the view incrementally as new CDC data arrives instead of recomputing everything. 

<button onclick="copyBlock()">Copy to clipboard</button>

<pre id="copy-block" style="font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; border:1px solid #e5e7eb; border-radius:10px; background:#f8fafc; padding:14px 16px; font-size:0.85rem; line-height:1.35; white-space:pre;">
<code>
<!-------------------ADD SOLUTION CODE BELOW------------------->
-- b. Create Gold Materialized View that lists all customers who have been removed (deleted) from the SCD Type 2 table.
CREATE OR REFRESH MATERIALIZED VIEW sdp_cdc_3_gold.removed_customers_gold_demo AS
SELECT
  customer_id,
  MAX_BY(name, __START_AT)      AS name,
  MAX_BY(address, __START_AT)   AS address,
  MAX_BY(city, __START_AT) AS city,
  MAX_BY(state, __START_AT)   AS state,
  MAX_BY(zip_code, __START_AT)   AS zip_code,
  MAX_BY(__START_AT, __START_AT) AS __START_AT,
  MAX_BY(__END_AT, __START_AT)  AS __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
GROUP BY customer_id
HAVING MAX_BY(__END_AT, __START_AT) IS NOT NULL;  -- Find the latest records __END_AT value and only return if not null
<!-------------------END SOLUTION CODE------------------->
</code></pre>

<script>
function copyBlock() {
  const el = document.getElementById("copy-block");
  if (!el) return;

  const text = el.innerText;

  // Preferred modern API
  if (navigator.clipboard && navigator.clipboard.writeText) {
    navigator.clipboard.writeText(text)
      .then(() => alert("Copied to clipboard"))
      .catch(err => {
        console.error("Clipboard write failed:", err);
        fallbackCopy(text);
      });
  } else {
    fallbackCopy(text);
  }
}

function fallbackCopy(text) {
  const textarea = document.createElement("textarea");
  textarea.value = text;
  textarea.style.position = "fixed";
  textarea.style.left = "-9999px";
  document.body.appendChild(textarea);
  textarea.select();
  try {
    document.execCommand("copy");
    alert("Copied to clipboard");
  } catch (err) {
    console.error("Fallback copy failed:", err);
    alert("Could not copy to clipboard. Please copy manually.");
  } finally {
    document.body.removeChild(textarea);
  }
}
</script>

### E3. Run the Final Spark Declarative Pipeline with the Materialized Views

1. **REQUIRED:** Since you're developing the pipeline, run it using **Run pipeline with full table refresh** to remove all checkpoints, truncates the tables, and runs the pipeline again.
    - (select the **Run Pipeline** dropdown arrow → **Run pipeline with full table refresh**).  

   This action will:  
   - Remove all existing checkpoints  
   - Drop previously created tables  
   - Restart the pipeline from scratch 

2. While it's running, update the bottom table view to show the **Incrementalization** column.  
   - **NOTE:** The **Incrementalization** column shows whether each materialized view was processed as a **Full recompute** or updated **incrementally**: [Incremental refresh for materialized views](https://docs.databricks.com/aws/en/optimizations/incremental-refresh).  
   
   <img src="./Includes/images/cdc_lecture/incremental_mvs_column2.png" alt="MV Incremental Column" width="1000"> 
<br></br>
3. After the run completes, verify the pipeline results match the example below.

#### Checkpoint

> **TROUBLESHOOTING:** If you see zero rows processed in your streaming tables, you likely ran a normal pipeline run instead of **Run pipeline with full table refresh**. A standard run will not reprocess data that was already ingested and processed.
<img src="./Includes/images/cdc_lecture/cdc_02_pipeline-run-with-mvs-one-file.png" alt="Pipeline Run 2 - MVs" width="1200">

### E4. Explore the Spark Declarative Pipeline Run

After the pipeline completes, review the outputs to confirm data flow and CDC behavior.

1. In the pipeline graph:
   - **939** rows were streamed through all streaming tables, passing Bronze data quality checks and upserting into **sdp_cdc_2_silver.customers_silver_scd2_demo**.
   - **sdp_cdc_3_gold.current_customers_gold_demo** shows **939** active customers, as no updates or deletes have occurred yet and all customers are active.
   - **sdp_cdc_3_gold.removed_customers_gold_demo** shows **0** rows since no customers have been removed.

2. In the **Lakeflow Pipeline Editor** table window:
   - The **Upserted records** column in **sdp_cdc_2_silver.customers_silver_scd2_demo** confirms all **939** rows were inserted as new customers.
   - Both Gold materialized views display `Full recompute` in the **Incrementalization** column because this was their first computation.

3. In the **Tables** view, select **customers_silver_scd2_demo**:
   - Preview the table to inspect the data.
   - Scroll right to see the **__START_AT** and **__END_AT** columns automatically added by the `AUTO CDC INTO` SCD Type 2 process.

## F. Incremental Data Processing
The final pipeline is built, let's add another file to cloud storage!

### F1. Land Additional Data
1. Before loading additional data, query the **sdp_cdc_2_silver.customers_silver_scd2_demo** streaming table for customers *23225* and *23617*.  

   Review the following details:

   - **customer_id = 23225**  
     - **Address:** `76814 Jacqueline Mountains Suite 815`  
     - **State:** `TX`  
     - **__END_AT:** `null`, indicating this is the current active record for the customer.

   - **customer_id = 23617**  
     - This record currently exists in the table and is active (**__END_AT** is `null`).


In [0]:
%sql
SELECT *
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
WHERE customer_id IN (23225, 23617);

2. Run the cell below to land a new JSON file (**01.json**) **in** your **customer_source_files** volume to simulate new files being added to your cloud storage location.

    Confirm that your **customer_source_files** volume contains two JSON files (**00.json and 01.json**).

In [0]:
## Copy a second JSON file into the user's volume
copy_files(
    copy_from = f'/Volumes/{your_marketplace_share_catalog_name}/v01/retail-pipeline/customers/stream_json', 
    copy_to = f'/Volumes/{my_catalog}/sdp_cdc_1_bronze/customer_source_files', 
    n = 2
)

## List files in your volume (confirm two files exist)
spark.sql(f'LIST "{my_vol_path}"').display()

3. Run the cell below to view the raw data in **01.json** before ingesting it into your pipeline.  

   Key points to notice:

   - The **01.json** file contains **23** rows.  
   
   - The **operation** column includes **UPDATE**, **DELETE**, and **NEW** values:
     - 12 customers marked as **UPDATE**  
     - 1 customer marked as **DELETE**  
     - 10 customers marked as **NEW**  

   - For **customer_id = 23225** (Sandy Adams):  
     - Original address (from **00.json**): `76814 Jacqueline Mountains Suite 815`, `TX`  
     - Updated address (in this file): `512 John Stravenue Suite 239`, `TN`  

   - For **customer_id = 23617**:  
     - The **operation** is **DELETE**.  
     - All other columns contain `NULL` values.


In [0]:
%sql
SELECT *
FROM read_files(
  my_vol_path || '/01.json',
  format => "JSON"
)
ORDER BY customer_id;

### F2. Process Updates and Deletes

1. Return to your **Spark Declarative Pipeline** and select **Run pipeline** to incrementally process the new JSON file (**01.json**) and apply SCD Type 2 logic to your Silver table.  

2. Once complete, verify the results match the example below.

### Checkpoint

<img src="./Includes/images/cdc_lecture/cdc_run_01json.png" alt="Pipeline Run 2" width="1200">


3. After running the pipeline, review the **Pipeline graph**:

   - **23 rows** were incrementally ingested from the new **01.json** file into both **customers_bronze_raw_demo** and **customers_bronze_clean_demo** , passing all data quality checks. 

   - In the **customers_silver_scd2_demo** streaming table (SCD Type 2), **35 rows** were processed and **upserted**.  
     - Breakdown of the **35** upserts from the **01.json** file:  
        - **12 updates** - each update adds a new version and marks the old one inactive (**24 total upserts**)  
        - **1 delete** - sets a value in the **__END_AT** column  
        - **10 new** customer inserts 

     **Total changes:** 24 + 1 + 10 = **35**

   - The **current_customers_gold_demo** materialized view contains **948** active customers.

   - The **removed_customers_gold_demo** materialized view contains **1** deleted customer.

## G. Pipeline Object Analysis and Validation

### G1. Query the CDC Silver Table

1. Query the data in the **sdp_cdc_2_silver.customers_silver_scd2_demo** streaming table with SCD Type 2 and observe the following:

   a. The table contains **961 rows**

      - **initial 939 customers**  
      - \+ **12 updates** to existing customers 
      - \+ **10 new customers**

   b. Scroll to the right and locate the **__END_AT** column. Then scroll down to rows **82** and **83**. Notice there are two rows for customer **22668**, the original record and the updated record.

**NOTE:** For demonstration purposes, many of the metadata columns were retained in the silver streaming table.

In [0]:
%sql
SELECT customer_id, address, name, __START_AT, __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
ORDER BY customer_id, __END_AT;

2. Run the query on the **sdp_cdc_2_silver.customers_silver_scd2_demo** streaming table for all rows where **__END_AT** `IS NOT NULL` to view all rows where those customers rows are now inactive.

Notice the following:
  - **13 rows** are returned (**12 UPDATES** + **1 DELETE**)
  - The **__END_AT** column indicates the date and time that the row was either updated or marked as removed.

In [0]:
%sql
SELECT customer_id, address, name, __START_AT, __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
WHERE __END_AT IS NOT NULL;

3. Query the **sdp_cdc_2_silver.customers_silver_scd2_demo** table for the **customer_id** *23225* (one of the customers that was updated). 

Notice the following:

- There are **two records** for that customer in the table.
- The original record from the **00.json** file now has a value in the **__END_AT** column, indicating that it is now inactive.
- The new record from the **01.json** file is now the active row and contains a `null` value in the **__END_AT** column.

In [0]:
%sql
SELECT customer_id, address, name, state, source_file, __START_AT, __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
WHERE customer_id = 23225;

4. In the **01.json** file, recall **customer_id** *23617* was marked as deleted. 

    Let's query the **sdp_cdc_2_silver.customers_silver_scd2_demo** table for that customer and view the results. 

    Notice that when a customer is marked as deleted, the **__END_AT** column contains the value of when that customer was deleted and became inactive, but the customer records STILL EXISTS in the **sdp_cdc_2_silver.customers_silver_scd2_demo** with SCD Type 2.

In [0]:
%sql
SELECT customer_id, address, name, __START_AT, __END_AT
FROM sdp_cdc_2_silver.customers_silver_scd2_demo
WHERE customer_id = 23617

### G2. Query the Current Customers Materialized View

1. To view your organization's most up-to-date customer data, you can query the materialized view **sdp_cdc_3_gold.current_customers_gold_demo**. 

    Remember, the query to create the materialized view filters for all **__END_AT** values that are `null` (active rows).

    Run the cell and view the results. Notice the following:
   - The current updated list of customers contains **948 rows**:
     - **939** from the initial file (**00.json**)
     - **+10** new customers from the update file (**01.json**)
     - **-1** deleted customer from the update file (**01.json**)
     - The table also contains the updated records from the **01.json** file.

In [0]:
%sql
SELECT customer_id, address, name, __START_AT, __END_AT, source_file
FROM sdp_cdc_3_gold.current_customers_gold_demo;

### G3. Query the Removed Customers Materialized View
1. To view your organization's deleted customers, you can query the materialized view **sdp_cdc_3_gold.removed_customers_gold_demo**. 

    Run the cell and view the results. Notice that there has been **1** customer marked as deleted (**customer_id** = *23617*) in the CDC silver table.

In [0]:
%sql
SELECT *
FROM sdp_cdc_3_gold.removed_customers_gold_demo;

## H. (Optional) Continue Incrementally Processing New Data

If you'd like to continue practicing, use the demonstration function `copy_files` to dynamically add another JSON file to your cloud storage location.  

**NOTE:** Ensure the variables you defined at the start of this lab: `your_marketplace_share_catalog_name`, `my_catalog` are still active for the function to work properly.

In [0]:
%skip

## Copy a third JSON file into the user's volume
copy_files(
    copy_from = f'/Volumes/{your_marketplace_share_catalog_name}/v01/retail-pipeline/customers/stream_json', 
    copy_to = f'/Volumes/{my_catalog}/sdp_cdc_1_bronze/customer_source_files', 
    n = 3   # <-- This value determines how many files should be in the volume. This will add the third json file 02.json to the volume
)

# Display the total files in the volume
spark.sql(f'LIST "{my_vol_path}"').display()

## I. Lab Clean Up
1. Feel free to delete the schemas you create in this demonstration by running cell below and confirming the delete (**Y**).

In [0]:
delete_schemas(
    catalog = my_catalog, ## <--- Your catalog name using the variable you set earlier
    schemas = ['sdp_cdc_1_bronze','sdp_cdc_2_silver','sdp_cdc_3_gold']
)

2. Delete your Spark Declarative Pipeline through the **Jobs & Pipelines** UI.

## J. Summary and Key Takeaways

You have successfully implemented an end-to-end **AUTO CDC for SCD Type 2** pipeline using **Lakeflow Spark Declarative Pipelines**. This demonstration showcased how modern data engineering can dramatically simplify complex Change Data Capture operations that traditionally required extensive manual coding.

**Simplified CDC Implementation:**
- Replaced complex manual `MERGE` operations with declarative `AUTO CDC INTO` syntax
- Automatic handling of late-arriving data using sequence columns
- Built-in SCD Type 2 logic without manual custom coding

**Robust Data Quality:**
- Multi-layered validation with conditional constraints for DELETE operations
- Flexible violation handling (WARN, DROP, FAIL) based on business requirements
- Comprehensive email validation using regex patterns

**Incremental Processing Efficiency:**
- Auto Loader for continuous file ingestion from cloud storage
- Incremental materialized view updates where possible
- Streaming architecture that processes only new changes